# 3. Coordinate-Based Meta-Analysis (ALE)

This notebook guides you through:
1. Loading your dataset with brain coordinates
2. Running ALE (Activation Likelihood Estimation) analysis
3. Applying multiple comparisons correction
4. Visualizing and exporting results

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from core import Study, Coordinate, MetaAnalysisDataset
from analysis.coordinate_based import ALEAnalysis

## Step 1: Load Dataset

Load the dataset created in notebook 02, or create a new one.

In [ ]:
# Try to load existing dataset
data_path = Path("../data/tmaze_dataset.json")

if data_path.exists():
    dataset = MetaAnalysisDataset.load(data_path)
    print(f"Loaded dataset: {dataset.name}")
else:
    # Create example dataset for demonstration
    dataset = MetaAnalysisDataset(
        name="Spatial Decision Meta-Analysis",
        description="Demo dataset for ALE analysis"
    )
    
    # Add sample studies with coordinates
    studies = [
        Study(
            study_id="smith2020",
            title="Neural correlates of spatial decision-making",
            authors=["Smith, J.", "Jones, M."],
            year=2020,
            n_total=30,
            coordinates=[
                Coordinate(x=-24, y=-8, z=52, region="Left premotor"),
                Coordinate(x=28, y=-10, z=48, region="Right premotor"),
                Coordinate(x=-6, y=12, z=44, region="Pre-SMA"),
            ]
        ),
        Study(
            study_id="johnson2019",
            title="Hippocampal activation during navigation",
            authors=["Johnson, A."],
            year=2019,
            n_total=25,
            coordinates=[
                Coordinate(x=-26, y=-20, z=-14, region="Left hippocampus"),
                Coordinate(x=28, y=-18, z=-12, region="Right hippocampus"),
                Coordinate(x=-4, y=-30, z=30, region="Posterior cingulate"),
            ]
        ),
        Study(
            study_id="chen2021",
            title="Prefrontal contributions to decision-making",
            authors=["Chen, L.", "Wang, X."],
            year=2021,
            n_total=35,
            coordinates=[
                Coordinate(x=-42, y=32, z=20, region="Left DLPFC"),
                Coordinate(x=44, y=30, z=22, region="Right DLPFC"),
                Coordinate(x=-2, y=28, z=40, region="ACC"),
            ]
        ),
        Study(
            study_id="garcia2022",
            title="Motor planning in spatial tasks",
            authors=["Garcia, R."],
            year=2022,
            n_total=28,
            coordinates=[
                Coordinate(x=-22, y=-6, z=54, region="Left premotor"),
                Coordinate(x=26, y=-8, z=50, region="Right premotor"),
                Coordinate(x=-38, y=-48, z=44, region="Left IPL"),
            ]
        ),
        Study(
            study_id="kim2023",
            title="Parietal cortex in spatial cognition",
            authors=["Kim, S.", "Lee, H."],
            year=2023,
            n_total=32,
            coordinates=[
                Coordinate(x=-32, y=-58, z=44, region="Left SPL"),
                Coordinate(x=34, y=-56, z=46, region="Right SPL"),
                Coordinate(x=-24, y=-70, z=40, region="Left precuneus"),
            ]
        )
    ]
    
    for study in studies:
        dataset.add_study(study)
    
    print("Created demo dataset")

print(dataset.summary())

In [ ]:
# View coordinates
coords_df = dataset.to_coordinates_df()
print(f"Total coordinates: {len(coords_df)}")
coords_df

## Step 2: Visualize Coordinate Distribution

Before running ALE, let's visualize where our coordinates are located.

In [ ]:
# Plot coordinate distributions
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].hist(coords_df['x'], bins=15, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('X (mm) - Left/Right')
axes[0].set_ylabel('Count')
axes[0].set_title('X Distribution')
axes[0].axvline(x=0, color='red', linestyle='--', alpha=0.5)

axes[1].hist(coords_df['y'], bins=15, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Y (mm) - Posterior/Anterior')
axes[1].set_title('Y Distribution')

axes[2].hist(coords_df['z'], bins=15, edgecolor='black', alpha=0.7)
axes[2].set_xlabel('Z (mm) - Inferior/Superior')
axes[2].set_title('Z Distribution')

plt.suptitle('Coordinate Distributions Across Studies', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# 2D scatter plots
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Sagittal view (X vs Z)
axes[0].scatter(coords_df['y'], coords_df['z'], alpha=0.6, s=50)
axes[0].set_xlabel('Y (mm)')
axes[0].set_ylabel('Z (mm)')
axes[0].set_title('Sagittal View (Y vs Z)')

# Coronal view (X vs Z)
axes[1].scatter(coords_df['x'], coords_df['z'], alpha=0.6, s=50)
axes[1].set_xlabel('X (mm)')
axes[1].set_ylabel('Z (mm)')
axes[1].set_title('Coronal View (X vs Z)')
axes[1].axvline(x=0, color='red', linestyle='--', alpha=0.3)

# Axial view (X vs Y)
axes[2].scatter(coords_df['x'], coords_df['y'], alpha=0.6, s=50)
axes[2].set_xlabel('X (mm)')
axes[2].set_ylabel('Y (mm)')
axes[2].set_title('Axial View (X vs Y)')
axes[2].axvline(x=0, color='red', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

## Step 3: Run ALE Analysis

ALE models each coordinate as a 3D Gaussian and computes the convergence across studies.

**Note:** This requires NiMARE to be installed (`pip install nimare`).

In [ ]:
# Initialize ALE analysis
try:
    ale = ALEAnalysis(dataset)
    print("ALE analysis initialized successfully")
    print(f"NiMARE version: {ale._nimare_version}")
except ImportError as e:
    print(f"Error: {e}")
    print("\nInstall NiMARE with: pip install nimare")
    ale = None

In [ ]:
# Run ALE analysis
if ale is not None:
    print("Running ALE analysis...")
    print("(This may take a few minutes for Monte Carlo correction)\n")
    
    results = ale.run(
        kernel_fwhm=None,          # Use sample-size based FWHM (recommended)
        null_method="approximate", # "approximate" (fast) or "montecarlo" (accurate)
        n_iters=1000,              # Increase for publication (10000+)
        correction_method="fwe",   # Family-wise error correction
        alpha=0.05,
        cluster_threshold=0.001
    )
    
    print("\nAnalysis complete!")
    print(f"\nSummary:")
    print(f"  Studies: {results['summary']['n_studies']}")
    print(f"  Coordinates: {results['summary']['n_coordinates']}")

## Step 4: Visualize Results

Create brain maps showing significant convergence.

In [ ]:
# Plot results on glass brain
if ale is not None and ale.corrected_results is not None:
    fig = ale.plot_results(
        display_mode="glass",
        title=f"ALE Results: {dataset.name}"
    )
    plt.show()

In [ ]:
# Plot orthogonal slices
if ale is not None and ale.corrected_results is not None:
    fig = ale.plot_results(
        display_mode="ortho",
        title="ALE Results - Orthogonal View"
    )
    plt.show()

In [ ]:
# Plot on anatomical slices with nilearn directly
if ale is not None:
    from nilearn import plotting
    
    # Get the Z-score map
    z_map = results.get('z_map')
    
    if z_map is not None:
        # Multiple axial slices
        fig = plt.figure(figsize=(14, 3))
        display = plotting.plot_stat_map(
            z_map,
            display_mode='z',
            cut_coords=[-10, 0, 20, 40, 50],
            title='ALE Z-scores (Axial Slices)',
            threshold=2.0,
            figure=fig
        )
        plt.show()

## Step 5: Cluster Table

Extract information about significant clusters.

In [ ]:
# Get cluster table
if ale is not None:
    cluster_table = results.get('cluster_table')
    
    if cluster_table is not None:
        print("Significant Clusters:")
        display(cluster_table)
    else:
        print("No significant clusters found at the specified threshold.")
        print("Try lowering the cluster threshold or using uncorrected results.")

## Step 6: Save Results

In [ ]:
# Create output directory
output_dir = Path("../results/ale")
output_dir.mkdir(parents=True, exist_ok=True)

if ale is not None and ale.results is not None:
    # Save NIfTI files
    ale.save_nifti(output_dir / "ale_zmap.nii.gz", map_type="z")
    print(f"Saved Z-map to {output_dir / 'ale_zmap.nii.gz'}")
    
    try:
        ale.save_nifti(output_dir / "ale_corrected.nii.gz", map_type="corrected")
        print(f"Saved corrected map to {output_dir / 'ale_corrected.nii.gz'}")
    except Exception:
        print("Could not save corrected map")
    
    # Save figure
    fig = ale.plot_results(display_mode="glass")
    fig.savefig(output_dir / "ale_glass_brain.png", dpi=300, bbox_inches='tight')
    print(f"Saved figure to {output_dir / 'ale_glass_brain.png'}")
    plt.close(fig)

In [ ]:
# Print summary
if ale is not None:
    print(ale.summary())

## Advanced: Compare Subgroups

You can run separate ALE analyses on subsets of studies and compare them.

In [ ]:
# Example: Split by year
early_studies = [s for s in dataset.studies if s.year and s.year < 2021]
late_studies = [s for s in dataset.studies if s.year and s.year >= 2021]

print(f"Early studies (<2021): {len(early_studies)}")
print(f"Later studies (>=2021): {len(late_studies)}")

# Note: Subgroup analysis typically requires more studies per group (10+) 
# to have adequate power

## Next Steps

1. **Add more studies** - ALE is more reliable with 10+ studies
2. **Increase iterations** - Use `n_iters=10000` for publication
3. **Try different corrections** - Compare FWE vs FDR results
4. **Subgroup analyses** - Compare patient groups, task types, etc.
5. **Proceed to notebook 04** for effect size meta-analysis